In [1]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 418.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 48.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=b5287927ff4bbade5bc090c17d27f390805200e2bc9a4eec0de2ff7f543cd707
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c

In [2]:
import torch

In [3]:
from ratsnlp.nlpbook.classification import ClassificationTrainArguments

In [5]:
args = ClassificationTrainArguments(
    pretrained_model_name = "beomi/kcbert-base",
    downstream_corpus_name = 'nsmc',
    downstream_corpus_root_dir = '/content/drive/MyDrive/class/language',
    downstream_model_dir = '/content/drive/MyDrive/class/language/checkpoint-doccls',
    batch_size = 32 if torch.cuda.is_available() else 4,
    learning_rate = 5e-5,
    max_seq_length = 128,
    epochs = 1,
    seed = 7    
)

In [6]:
from ratsnlp import nlpbook

In [7]:
nlpbook.set_seed(args)
nlpbook.set_logger(args)


INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/drive/MyDrive/class/language', downstream_model_dir='/content/drive/MyDrive/class/language/checkpoint-doccls', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=1, batch_size=32, cpu_workers=4, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/drive/MyDrive/class/language', downstream_model_dir='/content/drive/MyDrive/class/language/checkpoint-doccls', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_downl

set seed: 7


In [8]:
from Korpora import Korpora

In [9]:
Korpora.fetch(
    corpus_name = args.downstream_corpus_name,
    root_dir = args.downstream_corpus_root_dir,
    force_download=True
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 90.2MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 39.2MB/s]                            


In [10]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case = False
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

In [11]:
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset

In [12]:
corpus = NsmcCorpus()

In [ ]:
corpus.get_examples('/content/drive/MyDrive/class/language/nsmc', 'train')

In [ ]:
train_dataset= ClassificationDataset(
    args= args,
    corpus = corpus,
    tokenizer=tokenizer,
    mode='train'
)

In [ ]:
train_dataset.features

In [23]:
from torch.utils.data import DataLoader, RandomSampler

In [24]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size = args.batch_size,
    sampler = RandomSampler(train_dataset, replacement=False),
    collate_fn = nlpbook.data_collator,
    drop_last = False,
    num_workers= args.cpu_workers
)

In [25]:
from torch.utils.data import SequentialSampler

In [ ]:
val_dataset= ClassificationDataset(
    args= args,
    corpus = corpus,
    tokenizer=tokenizer,
    mode='test'
)

In [27]:
val_dataloader = DataLoader(
    val_dataset,
    batch_size = args.batch_size,
    sampler = SequentialSampler(val_dataset),
    collate_fn = nlpbook.data_collator,
    drop_last = False,
    num_workers= args.cpu_workers
)

In [28]:
from transformers import BertConfig, BertForSequenceClassification

In [29]:
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels = corpus.num_labels
)

In [30]:
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

In [31]:
from ratsnlp.nlpbook.classification import ClassificationTask

In [32]:
task = ClassificationTask(model, args) #classificationTask => 모델과 arg 가지구 최종 분류해줌!

In [33]:
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [34]:
trainer.fit(task, train_dataloaders = train_dataloader, val_dataloaders = val_dataloader)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.680   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [36]:
"""
  1. 설정 
  2. 데이터 불러오기 
  3. 토크나이저 불러오기 
  4. train 데이터 토큰화 -> train 데이터 로더
  5. valid 데이터 토큰화 -> valid 데이터 로더
  6. 모델 설정 불러오기 및 모델 불러오기
  7. 트레이너 만들기
  8. 학습
"""

'1. 설정 \n  2. 데이터 불러오기 \n  3. 토크나이저 불러오기 \n  4. train 데이터 토큰화 -> train 데이터 로더\n  5. test 데이터 토큰화 -> test 데이터 로더\n  6. 모델 설정\n  '